In [31]:
import pandas as pd


In [32]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Libaries')

In [33]:
# Importing self-made libaries
import sensitivity as st
import reload_lib as rl
import life_cycle_assessment as lc
import sensitivity_case1 as c1
import sensitivity_case2 as c2
import standards as s
import results_figures as rfig



lib = [st, lc, c1, c2, s,rfig]

In [34]:
# # Path to where the code is stored
# path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care'
# # Specifying the LCIA method
# lcia_method = 'recipe'

# ecoinevnt_paths = {'ev391apos' : r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_apos_ecoSpold02\datasets",
#                    'ev391consq' : r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_consequential_ecoSpold02\datasets",
#                    'ev391cutoff' : r"C:\Users\ruw\Downloads\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets"}

# system_path = [r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case1.xlsx', 
#                 r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case2.xlsx']

In [35]:
rl.reload_lib(lib)
path = r'C:/Users/ruw/Desktop'
path_github, ecoinevnt_paths, system_path = s.paths(path)

In [38]:
rl.reload_lib(lib)
variables = st.break_even_initialization(path_github, 'recipe')

case1 already exist
C:/Users/ruw/Desktop\RA\Single-use-vs-multi-use-in-health-care\results/case1
case1 already exist
C:/Users/ruw/Desktop\RA\Single-use-vs-multi-use-in-health-care\results/case1
case1 already exist
C:/Users/ruw/Desktop\RA\Single-use-vs-multi-use-in-health-care\results/case1
case2 already exist
C:/Users/ruw/Desktop\RA\Single-use-vs-multi-use-in-health-care\results/case2
case2 already exist
C:/Users/ruw/Desktop\RA\Single-use-vs-multi-use-in-health-care\results/case2
case2 already exist
C:/Users/ruw/Desktop\RA\Single-use-vs-multi-use-in-health-care\results/case2
Recipe is selected


In [39]:
case = 'case2'
file = s.join_path(path,'results')
file_path = s.join_path(file, f'{case}\\break_even_data_{case}.xlsx')

In [40]:
def df_index(key):
    if '1' in key:
        flow_leg = [
                    'H2S',
                    'H2R',
                    'ASC',
                    'ASW',
                    'H4S',
                    'H4R',
                    'ALC',
                    'ALW'
                    ]
        return flow_leg
    else:
        return ['SUD', 'MUD']
    

In [41]:
with pd.ExcelWriter(file_path) as writer:
    for key in variables.keys():
        df_GWP = variables[key][1]
        db_type = variables[key][2]
        database_name = variables[key][0]
        
        flow_legend = variables[key][4]
        columns = variables[key][-1]

        columns = lc.unique_elements_list(database_name)

        df_be, ignore = rfig.process_categorizing(df_GWP, case, flow_legend, columns)
        df_be.index = df_index(key)
        df_be_copy = rfig.break_even_orginization(df_be, database_name)
        

        # Write each DataFrame to a different sheet
        sheet_name = f"{key}"
        df_be_copy.to_excel(writer, sheet_name=sheet_name, index=True, header=True)